### Formulation
**Data**:
- $I$: the set of SKUs
- $J$: the set of aspects taking into consideration
- $W_j$: the weight put on each aspect $j$ (can be 0 to eliminate one particular aspect)
- $S_{ij}$: the scaled score for each SKU $i$ in each aspect $j$
- $B$: the set of brands
- $D$: the set of departments
- $F$: the set of flavors
- $B_{bi}$: whether or not SKU $i$ is brand $b$ (binary)
- $D_{di}$: whether or not SKU $i$ is in department $d$ (binary)
- $F_{fi}$: whether or not SKU $i$ is flavor $f$ (binary)
- $b_n$: current number of brands in the region chosen
- $d_n$: current number of departments in the region chosen
- $f_n$: current number of flvors in the region chosen
- $p_b$: allowance in upper and lower bounds for number of brands in the region chosen (difference)
- $p_d$: allowance in upper and lower bounds for number of departments in the region chosen (difference)
- $p_f$: allowance in upper and lower bounds for number of flavors in the region chosen (difference)
- excluded $m_i$: whether SKU $i$ exceed 95% of platform capacity (binary, 1 for exceed, 0 for not exceed)
- excluded $v_i$: whether SKU $i$ have extreme high (exceed 2 standard deviation above mean) inventory cost (defined as (safty stock) / (daily demand)) (binary, 1 for exceed, 0 for not exceed)
- $c$: the number of SKUs reserved to match competitor lineup
- $n$: the number of SKUs reserved to test new product

**Decision Variables**: For each SKU $i \in I$, let $x_i$ denotes whether we will choose this SKU or not for the region chosen. (Binary) 

**Formulation**:
$$\begin{aligned}
\text{Maximize} && \sum_{i \in I, j \in J} X_i W_j S_{ij} \\
\text{Subject to:} && \\
\text{(SKUs limit)} && \sum_{i \in I} X_i + c + n & \leq 250\\
\text{(Brand)}: && (1 - p_b) b_n \leq \sum_{b \in B} & (\text{if} \sum_{i \in I} B_{bi} = 0 \text{ then } 0, \text{if} \sum_{i \in I} B_{bi} = 1 \text{ then } 1 \text{ for $b \in B$}) \leq (1 + p_b) b_n \\
\text{(Department)}: && (1 - p_d) d_n \leq \sum_{d \in D} & (\text{if} \sum_{i \in I} D_{di} = 0 \text{ then } 0, \text{if} \sum_{i \in I} D_{di} = 1 \text{ then } 1 \text{ for $d \in D$}) \leq (1 + p_d) d_n \\
\text{(Flavor)}: && (1 - p_f) f_n \leq \sum_{f \in F} & (\text{if} \sum_{i \in I} F_{fi} = 0 \text{ then } 0, \text{if} \sum_{i \in I} F_{fi} = 1 \text{ then } 1 \text{ for $f \in F$}) \leq (1 + p_f) f_n \\
\text{(Binary)}: && x_i & \text{ is binary.} \\
\end{aligned}$$

**Auxiliary Decision Variables**:
- $Z_b$: whether or not brand $b$ is represented by the BDCs selected (binary)
- $Z_d$: whether or not department $d$ is represented by the BDCs selected (binary)
- $Z_f$: whether or not flavor $f$ is represented by the BDCs selected (binary)

**Linearized Formulation**:
$$\begin{aligned}
\text{Maximize} && \sum_{i \in I, j \in J} X_i W_j S_{ij} \\
\text{Subject to:} && \\
\text{(SKUs limit)} && \sum_{i \in I} X_i + c + n & \leq 250 \\
\text{($Z_b$ for Brand)}: && Z_b & \leq \sum_{i \in I} B_{bi} && \text{ for $b \in B$.} \\
\text{(Brand)}: && (1 - p_b) b_n & \leq \sum_{b \in B} Z_b \leq (1 + p_b) b_n \\
\text{($Z_d$ for Department)}: && Z_d & \leq \sum_{i \in I} D_{di} && \text{ for $d \in D$.} \\
\text{(Department)}: && (1 - p_d) d_n & \leq \sum_{d \in D} Z_d \leq (1 + p_d) d_n \\
\text{($Z_f$ for Flavor)}: && Z_f & \leq \sum_{i \in I} F_{fi} && \text{ for $f \in F$.} \\
\text{(Flavor)}: && (1 - p_f) f_n & \leq \sum_{f \in F} Z_f \leq (1 + p_f) f_n \\
\text{(Binary)}: && x_i, Z_b, Z_d, Z_f & \text{ is binary.} \\
\end{aligned}$$

In [123]:
import pandas as pd
import gurobipy as grb

In [15]:
p_b = 0.1
p_d = 0.2
p_f = 0.3
c = 10
n = 20
region = 'SOUTHWEST'
SKUlimit = 250

In [13]:
scaled = pd.read_csv('inputs/scaled_data.csv', index_col = 0)
weight = pd.read_excel('inputs/weight.xlsx', index_col = 0)
constr_brand = pd.read_excel('inputs/Constraints.xlsx', sheet_name = 'number_of_brand', index_col = 0)
constr_dept = pd.read_excel('inputs/Constraints.xlsx', sheet_name = 'number_of_dept', index_col = 0)
constr_flavor = pd.read_excel('inputs/Constraints.xlsx', sheet_name = 'number_of_flavor', index_col = 0)

brand_dum = pd.read_csv('inputs/brand.csv', index_col = 0)
brand_dum.index = brand_dum['BDC']
del brand_dum['BDC']

department_dum = pd.read_csv('inputs/department.csv', index_col = 0)
department_dum.index = department_dum['BDC']
del department_dum['BDC']

flavor_dum = pd.read_csv('inputs/flavor.csv', index_col = 0)
flavor_dum.index = flavor_dum['BDC']
del flavor_dum['BDC']

In [35]:
scaled.head()

,sales,shelf_life,production_cost,estimated_margins,shipping_cost,inventory_cost,return_rate
BDC,,,,,,,
1005006,0.115499,0.408163,0.308284,10.0,0.124009,0.257668,0.164460
1005007,0.026674,0.408163,0.308284,10.0,0.006762,0.308320,0.111599
1005011,0.179863,0.408163,0.308284,10.0,0.094568,0.188354,0.338483
1005182,0.023810,0.408163,0.308284,10.0,0.035472,0.025623,0.219408
1009011,0.004990,0.408163,0.346212,10.0,0.205864,1.254425,0.956636


In [56]:
mod = grb.Model()

I = scaled.index
J = scaled.columns
B = brand_dum.columns
D = department_dum.columns
F = flavor_dum.columns

x = mod.addVars(I, name = 'BDC', vtype = grb.GRB.BINARY)
zb = mod.addVars(B, name = 'brand', vtype = grb.GRB.BINARY)
zd = mod.addVars(D, name = 'department', vtype = grb.GRB.BINARY)
zf = mod.addVars(F, name = 'flaAvor', vtype = grb.GRB.BINARY)

mod.setObjective(sum(x[i] * weight.loc[j, 'weight'] * scaled.loc[i, j] for i in I for j in J), sense = grb.GRB.MAXIMIZE)

mod.addConstr(sum(x[i] for i in I) + c + n <= SKUlimit)
for b in B:
    mod.addConstr(zb[b] <= sum(brand_dum.loc[i, b] for i in I))
mod.addConstr((1 - p_b) * constr_brand.loc[region, 'brand'] <= sum(zb[b] for b in B) <= (1 + p_b) * constr_brand.loc[region, 'brand'])
for d in D:
    mod.addConstr(zd[d] <= sum(department_dum.loc[i, d] for i in I))
mod.addConstr((1 - p_d) * constr_dept.loc[region, 'department'] <= sum(zd[d] for d in D) <= (1 + p_d) * constr_dept.loc[region, 'department'])
for f in F:
    mod.addConstr(zf[f] <= sum(flavor_dum.loc[i, f] for i in I))
mod.addConstr((1 - p_f) * constr_flavor.loc[region, 'flavor'] <= sum(zf[f] for f in F) <= (1 + p_f) * constr_flavor.loc[region, 'flavor'])

mod.setParam('OutputFlag', False)
mod.optimize()

In [116]:
score = []
for i in I:
    for j in J:
        score.append([i, j, weight.loc[j, 'weight'] * scaled.loc[i, j]])
score = pd.DataFrame(score, columns = ['BDC', 'aspect', 'score'])
score2 = pd.DataFrame(score.groupby(['BDC']).sum())
score2['BDC'] = score2.index
score2 = score2.reset_index(drop = True)

In [117]:
score2.head()

,score,BDC
0,3.115050,1005006
1,3.066166,1005007
2,3.145166,1005011
3,3.067627,1005182
4,3.142369,1009011


In [118]:
output = []

for i in I:
    if x[i].x == 1:
        dec = 'Yes'
    else:
        dec = 'No'
    output.append([i, dec])

output = pd.DataFrame(output, columns = ['BDC', 'decision'])
output['score'] = score2['score']

In [119]:
output.head()

,BDC,decision,score
0,1005006,No,3.115050
1,1005007,No,3.066166
2,1005011,Yes,3.145166
3,1005182,No,3.067627
4,1009011,Yes,3.142369


In [120]:
output.sort_values('score', ascending = False).head()

,BDC,decision,score
277,15016033,Yes,8.345181
316,20016097,Yes,6.601742
96,3015009,Yes,5.755658
273,15016008,Yes,5.333594
290,15021033,Yes,5.279045


In [122]:
output.to_csv('output.csv', index = False)

In [ ]:
import pandas as pd
import numpy as np
import gurobipy as grb

# read and merge raw data
estimated_margins=pd.read_csv('project_data/estimated_margins.csv')
inventory_data = pd.read_excel('project_data/2018_P12_inventory_report.xlsx',skiprows=2)
brand_BDC = estimated_margins.merge(inventory_data, on = "BDC")
distribution_cost = pd.read_excel('project_data/distribution_cost_cleaned.xlsx',sheet_name = "shipment_quantities")
Dept_BDC = distribution_cost.merge(inventory_data, on = "BDC")

# read defined inputs
scaled = pd.read_csv('inputs/scaled_data.csv', index_col = 0)
weight = pd.read_excel('inputs/weight.xlsx', index_col = 0)

# count number of brands, departments and flavors in each region
# brand
number_BDC = brand_BDC.groupby("region")["brand"].nunique()
constr_brand = pd.DataFrame(number_BDC)
# department
number_dept = Dept_BDC.groupby("region")["department"].nunique()
constr_dept = pd.DataFrame(number_dept)
# flavor
number_flavor = Dept_BDC.groupby("region")["flavor"].nunique()
constr_flavor = pd.DataFrame(number_flavor)

# create dummy variables for brands, departments and flavors for each SKU
# brand
brand = estimated_margins[['BDC', 'brand']].drop_duplicates()
brand_dum = pd.get_dummies(brand, prefix='brand_', columns=['brand'])
brand_dum.index = brand_dum['BDC']
del brand_dum['BDC']
# department
department = distribution_cost[['BDC', 'department']].drop_duplicates()
department_dum = pd.get_dummies(department, prefix = 'department_', columns = ['department'])
department_dum.index = department_dum['BDC']
del department_dum['BDC']
# flavor
flavor = distribution_cost[['BDC', 'flavor']].drop_duplicates()
flavor_dum = pd.get_dummies(flavor, prefix = 'flavor_', columns = ['flavor'])
flavor_dum.index = flavor_dum['BDC']
del flavor_dum['BDC']

# parameters
p_b = 0.1
p_d = 0.2
p_f = 0.3
c = 10
n = 20
region = 'SOUTHWEST'
SKUlimit = 250

# optimization model
mod = grb.Model()

# data
I = scaled.index
J = scaled.columns
B = brand_dum.columns
D = department_dum.columns
F = flavor_dum.columns

# add variables
x = mod.addVars(I, name = 'BDC', vtype = grb.GRB.BINARY)
zb = mod.addVars(B, name = 'brand', vtype = grb.GRB.BINARY)
zd = mod.addVars(D, name = 'department', vtype = grb.GRB.BINARY)
zf = mod.addVars(F, name = 'flaAvor', vtype = grb.GRB.BINARY)

# set objective
mod.setObjective(sum(x[i] * weight.loc[j, 'weight'] * scaled.loc[i, j] for i in I for j in J), sense = grb.GRB.MAXIMIZE)

# add constraints
mod.addConstr(sum(x[i] for i in I) + c + n <= SKUlimit)
for b in B:
    mod.addConstr(zb[b] <= sum(brand_dum.loc[i, b] for i in I))
mod.addConstr((1 - p_b) * constr_brand.loc[region, 'brand'] <= sum(zb[b] for b in B) <= (1 + p_b) * constr_brand.loc[region, 'brand'])
for d in D:
    mod.addConstr(zd[d] <= sum(department_dum.loc[i, d] for i in I))
mod.addConstr((1 - p_d) * constr_dept.loc[region, 'department'] <= sum(zd[d] for d in D) <= (1 + p_d) * constr_dept.loc[region, 'department'])
for f in F:
    mod.addConstr(zf[f] <= sum(flavor_dum.loc[i, f] for i in I))
mod.addConstr((1 - p_f) * constr_flavor.loc[region, 'flavor'] <= sum(zf[f] for f in F) <= (1 + p_f) * constr_flavor.loc[region, 'flavor'])

# optimization
mod.setParam('OutputFlag', False)
mod.optimize()

# generate output
# compute score
score = []
for i in I:
    for j in J:
        score.append([i, j, weight.loc[j, 'weight'] * scaled.loc[i, j]])
score = pd.DataFrame(score, columns = ['BDC', 'aspect', 'score'])
score2 = pd.DataFrame(score.groupby(['BDC']).sum())
score2['BDC'] = score2.index
score2 = score2.reset_index(drop = True)
# combine decision and score
output = []
for i in I:
    if x[i].x == 1:
        dec = 'Yes'
    else:
        dec = 'No'
    output.append([i, dec])

output = pd.DataFrame(output, columns = ['BDC', 'decision'])
output['score'] = score2['score']

In [ ]:
SKUselector(
    estimated_margins = 'project_data/estimated_margins.csv',
    inventory_data = 'project_data/2018_P12_inventory_report.xlsx',
    distribution_cost = 'project_data/distribution_cost_cleaned.xlsx',
    scaled_data = 'inputs/scaled_data.csv',
    weight = 'inputs/weight.xlsx',
    parameter = 'inputs/parameter.xlsx',
    OutputFile = 'output.csv')

In [ ]:
def SKUselector(estimated_margins, inventory_data, distribution_cost, scaled_data, weight, parameter, OutputFile):
    import pandas as pd
    import gurobipy as grb
    
    # read and merge raw data
    estimated_margins = pd.read_csv(estimated_margins)
    inventory_data = pd.read_excel(inventory_data, skiprows=2)
    brand_BDC = estimated_margins.merge(inventory_data, on = "BDC")
    distribution_cost = pd.read_excel(distribution_cost, sheet_name = "shipment_quantities")
    Dept_BDC = distribution_cost.merge(inventory_data, on = "BDC")

    # read defined inputs
    scaled = pd.read_csv(scaled_data, index_col = 0)
    weight = pd.read_excel(weight, index_col = 0)

    # read defined parameters
    parameter = pd.read_excel(parameter)
    region = parameter.loc['Region', 'parameter']
    SKUlimit = parameter.loc['SKU limit', 'parameter']
    p_b = parameter.loc['Brand allowance', 'parameter']
    p_d = parameter.loc['Department allowance', 'parameter']
    p_f = parameter.loc['Flavor allowance', 'parameter']
    c = parameter.loc['Number of SKUs reserved for match competitor linup', 'parameter']
    n = parameter.loc['Number of SKUs reserved for new product testing', 'parameter']

    # count number of brands, departments and flavors in each region
    # brand
    number_BDC = brand_BDC.groupby("region")["brand"].nunique()
    constr_brand = pd.DataFrame(number_BDC)
    # department
    number_dept = Dept_BDC.groupby("region")["department"].nunique()
    constr_dept = pd.DataFrame(number_dept)
    # flavor
    number_flavor = Dept_BDC.groupby("region")["flavor"].nunique()
    constr_flavor = pd.DataFrame(number_flavor)

    # create dummy variables for brands, departments and flavors for each SKU
    # brand
    brand = estimated_margins[['BDC', 'brand']].drop_duplicates()
    brand_dum = pd.get_dummies(brand, prefix='brand_', columns=['brand'])
    brand_dum.index = brand_dum['BDC']
    del brand_dum['BDC']
    # department
    department = distribution_cost[['BDC', 'department']].drop_duplicates()
    department_dum = pd.get_dummies(department, prefix = 'department_', columns = ['department'])
    department_dum.index = department_dum['BDC']
    del department_dum['BDC']
    # flavor
    flavor = distribution_cost[['BDC', 'flavor']].drop_duplicates()
    flavor_dum = pd.get_dummies(flavor, prefix = 'flavor_', columns = ['flavor'])
    flavor_dum.index = flavor_dum['BDC']
    del flavor_dum['BDC']

    # optimization model
    mod = grb.Model()

    # data
    I = scaled.index
    J = scaled.columns
    B = brand_dum.columns
    D = department_dum.columns
    F = flavor_dum.columns

    # add variables
    x = mod.addVars(I, name = 'BDC', vtype = grb.GRB.BINARY)
    zb = mod.addVars(B, name = 'brand', vtype = grb.GRB.BINARY)
    zd = mod.addVars(D, name = 'department', vtype = grb.GRB.BINARY)
    zf = mod.addVars(F, name = 'flaAvor', vtype = grb.GRB.BINARY)

    # set objective
    mod.setObjective(sum(x[i] * weight.loc[j, 'weight'] * scaled.loc[i, j] for i in I for j in J), sense = grb.GRB.MAXIMIZE)

    # add constraints
    mod.addConstr(sum(x[i] for i in I) + c + n <= SKUlimit)
    for b in B:
        mod.addConstr(zb[b] <= sum(brand_dum.loc[i, b] for i in I))
    mod.addConstr((1 - p_b) * constr_brand.loc[region, 'brand'] <= sum(zb[b] for b in B) <= (1 + p_b) * constr_brand.loc[region, 'brand'])
    for d in D:
        mod.addConstr(zd[d] <= sum(department_dum.loc[i, d] for i in I))
    mod.addConstr((1 - p_d) * constr_dept.loc[region, 'department'] <= sum(zd[d] for d in D) <= (1 + p_d) * constr_dept.loc[region, 'department'])
    for f in F:
        mod.addConstr(zf[f] <= sum(flavor_dum.loc[i, f] for i in I))
    mod.addConstr((1 - p_f) * constr_flavor.loc[region, 'flavor'] <= sum(zf[f] for f in F) <= (1 + p_f) * constr_flavor.loc[region, 'flavor'])

    # optimization
    mod.setParam('OutputFlag', False)
    mod.optimize()

    # generate output
    # compute score
    score = []
    for i in I:
        for j in J:
            score.append([i, j, weight.loc[j, 'weight'] * scaled.loc[i, j]])
    score = pd.DataFrame(score, columns = ['BDC', 'aspect', 'score'])
    score2 = pd.DataFrame(score.groupby(['BDC']).sum())
    score2['BDC'] = score2.index
    score2 = score2.reset_index(drop = True)
    # combine decision and score
    output = []
    for i in I:
        if x[i].x == 1:
            dec = 'Yes'
        else:
            dec = 'No'
        output.append([i, dec])

    output = pd.DataFrame(output, columns = ['BDC', 'decision'])
    output['score'] = score2['score']

    output.to_csv(OutputFile)